# Libraries

In [2]:
import numpy as np
import sympy as sp
import sympy.physics.quantum as sq
import sympy.physics.quantum.qubit as sqq
import sympy.physics.quantum.gate as sqg

import cvxpy as cp

import matplotlib.pyplot as plt
import networkx as nx

sp.init_printing(use_latex='mathjax')
from IPython.display import display

## idealFid

In [3]:
class idealFid:
    
    def __init__(s):
        P1 = s.proj(1,5)
        P2 = s.proj(2,5)
        P3 = s.proj(3,5)
        P4 = s.proj(4,5)
        P5 = s.proj(5,5)
        s.P = [P1,P2,P3,P4,P5]
        s.terms = 5

    def proj(s,j,n):
        pi=np.pi
        temp1 = np.cos(pi/n)/(1+np.cos(pi/n))
        temp2 = 1-temp1
        temp3 = j*pi*(n-1)/n
        vec = np.asarray([np.sqrt(temp1),np.sqrt(temp2)*np.sin(temp3), np.sqrt(temp2)*np.cos(temp3)])
        mat = np.outer(vec,vec)
        return mat
    
    
    def idealVal(s,projList):
        op = s.P[projList[0]]
        for iProj in projList[1:]:
            op = op@s.P[iProj]
        return op[0,0]
    

## Extending the braket library to include orthogonality

In [4]:
class onStruct:
    G = nx.Graph()

class onKet(sq.Ket,onStruct):
    def _eval_innerproduct(self,bra,**hints):
        #Check if the innerproduct is with yourself, then set 1
        if sq.Dagger(bra) == self:
            return sp.Integer(1)
        #Else, if the innerproduct is with a neighbor from the graph, set 0
        elif sq.Dagger(bra) in G[self]:
            return sp.Integer(0)
        #Else, do nothing; not enough information
    
    @classmethod
    def set_on_rel(cls,givenG):
        cls.G=givenG
    
    @classmethod
    def dual_class(self):
        return onBra

    
class onBra(sq.Bra,onStruct):
    @classmethod
    def dual_class(self):
        return onKet

In [5]:
class bKet(sq.Ket):
    
    #total number of kets (automatically increases as new instances are initialised)
    totalKets=0
    
    #this method automates the increase
    @classmethod
    def _eval_args(cls,args):
        #validate input type
        if not isinstance(args[0],int):
            raise ValueError("Integer expected in the argument, got: %r"%args[0])

        #if a ket |5> is initialised, the dimension is assumed to be 6 since we have |0>,|1>,...|5>
        if int(args[0])+1 > cls.totalKets:
            cls.totalKets=args[0]+1
        
        #pass control back to the base class
        return sq.Ket._eval_args(args) 
    
    #Could not find a way of using just one function; need to go through the two
    #based on a combination of the qubit implementation and the 
    def _represent_default_basis(s,**options):
        return s._represent_bKet(None,**options) #syntax taken from the Qubit library
    
    def _represent_bKet(s,basis,**options): #_represent_default_basis
        a=np.zeros(s.__class__.totalKets,dtype=int)
        a[s.label[0]]=1
        return sp.Matrix(a)
    
    @classmethod
    def set_dimension(cls,arg):
        cls.totalKets=arg
    
    def _eval_innerproduct(self,bra,**hints):
        #Check if the innerproduct is with yourself, then set 1
        if sq.Dagger(bra) == self:
            return sp.Integer(1)
        #Else, if the innerproduct is with a neighbor from the graph, set 0
        else:
            return sp.Integer(0)
        #Else, do nothing; not enough information
    

    @classmethod
    def dual_class(self):
        return bBra

    
class bBra(sq.Bra):
    # @classmethod
    # def _eval_args(cls,args):
    #     return args
    
    @classmethod
    def dual_class(cls):
        return bKet

### Debugging | representation example

In [ ]:
class SzUpKet(sq.Ket):
    def _represent_SzUpKet(s,basis,**options):
        return sp.Matrix([1,0])
    
#     def _represent_SzOp(s,basis,**options):
#         return sp.Matrix([1,0])

class SzOp(sq.Operator):
    pass

sz = SzOp('Sz')
up = SzUpKet('up')
sq.represent(up,basis=SzUpKet) #,basis=sz)

### Testing | Dimension increasing?

In [ ]:
# class bKet(sq.Ket):
    
#     #total number of kets (automatically increases as new instances are initialised)
#     totalKets=0
    
#     #this method automates the increase
#     @classmethod
#     def _eval_args(cls,args):
#         #validate input type
#         if not isinstance(args[0],int):
#             raise ValueError("Integer expected in the argument, got: %r"%args[0])

#         #if a ket |5> is initialised, the dimension is assumed to be 6 since we have |0>,|1>,...|5>
#         if int(args[0])+1 > cls.totalKets:
#             cls.totalKets=args[0]+1
        
#         #pass control back to the base class
#         return sq.Ket._eval_args(args) 
    
#     #Could not find a way of using just one function; need to go through the two
#     #based on a combination of the qubit implementation and the 
#     def _represent_default_basis(s,**options):
#         return s._represent_bKet(None,**options) #syntax taken from the Qubit library
    
#     def _represent_bKet(s,basis,**options): #_represent_default_basis
#         a=np.zeros(s.__class__.totalKets,dtype=int)
#         a[s.label[0]]=1
#         return sp.Matrix(a)
    
#     @classmethod
#     def set_dimension(cls,arg):
#         cls.totalKets=arg
    
#     def _eval_innerproduct(self,bra,**hints):
#         #Check if the innerproduct is with yourself, then set 1
#         if sq.Dagger(bra) == self:
#             return sp.Integer(1)
#         #Else, if the innerproduct is with a neighbor from the graph, set 0
#         else:
#             return sp.Integer(0)
#         #Else, do nothing; not enough information
    

#     @classmethod
#     def dual_class(self):
#         return bBra

    
# class bBra(sq.Bra):
#     # @classmethod
#     # def _eval_args(cls,args):
#     #     return args
    
#     @classmethod
#     def dual_class(cls):
#         return bKet

In [ ]:
a=bKet(0)

b=a*sq.Dagger(a)

print(bKet.totalKets)
sq.represent(a)

c=bKet(3);
sq.represent(a)
sq.represent(b)
sq.represent(c)

In [ ]:
a=bKet(0); a

In [ ]:
sq.represent(a)

In [ ]:
b=bKet(1); b

In [ ]:
sq.represent(b)

In [ ]:
sq.represent(a)

In [ ]:
c=bKet(3); c

In [ ]:
sq.represent(a)

## Extending the tensor product functionality—tsimp and tdsimp

In [6]:

def powerDrop(expr):
    if isinstance(expr,sp.Pow): #TODO: make sure the base is not too complex
        # print("PowerEncountered")
        if expr.exp>=2:
            # print("glaba")
            # display(expr.base)
            _=sq.qapply(sp.Mul(expr.base,expr.base))
            if expr.exp>2:
                return powerDrop(_*sp.Pow(expr.base,expr.exp-2))
            else:
                return _
        else:
            return expr #autoDropDim(sp.Mul(expr.base,expr.base))
    else:
        if expr.has(sp.Pow):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add) or isinstance(expr,sq.TensorProduct):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(powerDrop(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                elif isinstance(expr,sq.TensorProduct):
                    return sq.TensorProduct(*new_args)  

            else:
                return expr
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        
    
def autoDropDim(expr):
    #print("Expression")
    #if isinstance(expr,sp.Mul):
        #print("type:multiplier")
    #display(expr)
    
    
    if isinstance(expr,sq.TensorProduct):
        new_args=[]
        for _ in expr.args:
            #display(_)
            #print(type(_))
            if _ != sp.Integer(1):
            #if not isinstance(_,core.numbers.One):
                new_args.append(_)
        #print("TensorProduct with %d non-ones in the tensor product"%len(new_args))
        if(len(new_args)==0):
            return sp.Integer(1)
        else:
            return sq.TensorProduct(*new_args)
    else:
        if expr.has(sq.TensorProduct):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(autoDropDim(_))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr #when the expression is just an integer or some such


        
def tsimp(e,pruneMe=True):
    res=sq.qapply(powerDrop(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit()))
    if pruneMe:
        return prune(res)
    else:
        return res

def tdsimp(e,pruneMe=True):
    res=autoDropDim(sq.qapply(powerDrop(autoDropDim(sq.tensorproduct.tensor_product_simp(sq.qapply(e)).doit()))))
    if pruneMe:
        return prune(res)
    else:
        return res
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(e).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp_Mul(sq.qapply(e)).doit())
    #return autoDropDim(sq.tensorproduct.tensor_product_simp(e).doit())

## Find Coeff was not reliable

In [7]:
# depth=0
# depthThresh=20

def findCoeff(obj,lett):
#     global depth
    
#     print("Parent object:")
#     display(obj)
    
    if(obj==None):
#         print("coefficent is zero")
        return 0
    elif not (isinstance(obj,sp.Mul) or isinstance(obj,sp.Add)):        
        #the coefficient may be one but we couldn't see it earlier
        return obj.coeff(lett)
    else:
        #try to find the coefficient directly        
        result=obj.coeff(lett)
        #Did not work? 
        if(result==0):
            #try for each segment
            for _ in obj.args:
#                 print("child:")
#                 display(_)
                result=_.coeff(lett)
#                 print("coefficient of child:",result)
                
                #still did not work?
                if(result==0):
                    #try recursing
                    result = findCoeff(_,lett)    
                
                #found? Stop searching
                if(result!=0):
                    break
                    #return result

        #return whatever was found | worked
        return result



## Prune

In [8]:
def prune(expr,thr=10,remNum=False):
    if isinstance(expr,sp.Number): 
        if remNum==False:
            if sp.Abs(expr)<10**(-thr):
                return sp.Integer(0)
            else:
                return expr
        else:
            return sp.Integer(1)
    else:
        if expr.has(sp.Number):
            #if it is a sum or a product, run this function for each part and then combine the parts; return the result
            if isinstance(expr,sp.Mul) or isinstance(expr,sp.Add) or isinstance(expr,sq.TensorProduct):
                new_args=[] #list(expr.args)
                for _ in expr.args:
                    new_args.append(prune(_,thr,remNum))
                if isinstance(expr,sp.Mul):        
                    return sp.Mul(*new_args)
                elif isinstance(expr,sp.Add):
                    return sp.Add(*new_args)  
                elif isinstance(expr,sq.TensorProduct):
                    return sq.TensorProduct(*new_args)  

            else:
                return expr
            #There would be no else here because tensor product simp would have removed that part
        else:
            return expr        

# test=(A[0]*2)
# test.has(sp.Number)
# prune(test,remNum=True)

# Swap KCBS | Playfield 

## Inputs

In [ ]:
class inputs:
    graphSize = 5
    p = 1/np.sqrt(5)*np.array([1,1,1,1,1])

## The Sympy Part | playground

In [ ]:
k0 = sq.Ket('φ')
k1 = sq.Ket('ψ')
superposition = k0+k1; superposition

In [ ]:
inner_prod = sq.Dagger(superposition)*superposition; inner_prod

In [ ]:
sq.qapply(inner_prod)

In [ ]:
density = sq.qapply(superposition*sq.Dagger(superposition)); density

In [ ]:
qubit_state = sqq.Qubit('00'); qubit_state

In [ ]:
isinstance(qubit_state,sq.State)

In [ ]:
gate_operator = sqg.HadamardGate(0); gate_operator

In [ ]:
isinstance(gate_operator,sq.Operator)

In [ ]:

a=sqq.Qubit(0)
b=sqq.Qubit(1)

d = sq.qapply(sq.Dagger(a)*b).doit(); d

### Playing with classes

In [ ]:
class A:
    gla=1

class B:
    glaba=3
    
class C(B,A):
    glow=4

c=C()

print(isinstance(c,A),isinstance(c,B),isinstance(c,C))


### Kishor meets the class

In [ ]:
class Person:
    name="blank"
    
    def set_name(self,arg):
        self.name = arg
    
    @property
    def myName(self):
        return self.name

    def get_name(self):
        return self.name
    
Kishor = Person() 
Kishor.set_name("Bharti")

M = Person()
M.set_name("M")



print(M.myName, Kishor.myName, M.get_name() )

print(M.myName,Kishor.myName)

In [ ]:
class Person:
    name="blank"
    compatibility=""
    
    def set_name(self,arg):
        self.name = arg
    
    @property
    def myName(self):
        return self.name
    
    @classmethod
    def set_comp(cls,arg):
        cls.compatibility=arg
        
    @classmethod
    def get_comp(cls):
        return cls.compatibility
        
    @property
    def comp(self):
        return self.compatibility
    
Kishor = Person() 
Kishor.set_name("Bharti")

M = Person()
M.set_name("M")

Person.set_comp("M hates K")
print(Person.get_comp())

print(M.myName,Kishor.myName)

print(M.comp,Kishor.comp)

In [ ]:
class A:
    aVal="blank"
    
    @property
    def myVal(self):
        return self.aVal

class C:
    cVal="blank3"
    
    @property
    def myVal3(self):
        return self.cVal
    
class B(A,C):
    bVal="blank2"
    
    @property
    def myVal2(self):
        return self.bVal
    
a = B()

print(a.myVal2,a.myVal,a.myVal3)

### Preliminary implementation of orthogonormal kets (uses labels)

In [ ]:
class onKet(sq.Ket):
#     a=1
    
    #@classmethod
    #this @classmethod descriptor was killing it
    #it was sending the class in "self" instead of the instance in "self"
    #makes sense now
    #I can perhaps use this to set orthogonality relations a little more generally
    def _eval_innerproduct(self,bra,**hints):
        #print(self.label,bra.label)
        if self.label == bra.label:
            return sp.Integer(1)
        else:
            return sp.Integer(0)
    
    @classmethod
    def dual_class(self):
        return onBra
    @property
    def iAm(self):
        return self.label
    
class onBra(sq.Bra):
#     a=2

#     @classmethod
#     def _eval_innerproduct(self,ket,**hints):
#         if self.label == ket.label:
#             return sp.Integer(1)
#         else:
#             return sp.Integer(0)
    
    @classmethod
    def dual_class(self):
        return onKet

In [ ]:
a=onKet(0)
b=onKet(1)

In [ ]:
c=a+b; c

In [ ]:
sq.qapply(sq.Dagger(b)*a).doit()

In [ ]:
d=sq.Dagger(c)*b
sq.qapply(d).doit()

### Orthogonal Ket, general orthogonality relation

In [ ]:
class onStruct:
    G = nx.Graph()

class onKet(sq.Ket,onStruct):
#     a=1
    #@classmethod
    #this @classmethod descriptor was killing it
    #it was sending the class in "self" instead of the instance in "self"
    #makes sense now
    #I can perhaps use this to set orthogonality relations a little more generally
    def _eval_innerproduct(self,bra,**hints):
        #print(self.label,bra.label)        
        if sq.Dagger(bra) == self:
        #if self.label == bra.label:
            return sp.Integer(1)
        elif sq.Dagger(bra) in G[self]:
            return sp.Integer(0)
    
    @classmethod
    def set_on_rel(cls,givenG):
        cls.G=givenG
    
    @classmethod
    def dual_class(self):
        return onBra

    
class onBra(sq.Bra,onStruct):
#     a=2

#     @classmethod
#     def _eval_innerproduct(self,ket,**hints):
#         if self.label == ket.label:
#             return sp.Integer(1)
#         else:
#             return sp.Integer(0)    
    @classmethod
    def dual_class(self):
        return onKet

In [ ]:
#define the kets
a=onKet(1)
b=onKet(2)
c=onKet(3)
d=onKet(4)

#define the orthogonality relation using a graph (edges mean orthogonal)
G = nx.Graph()
G.add_nodes_from([a,b,c,d])
G.add_edges_from([[a,b],[b,c],[c,d],[d,a]])

#Add this graph to the class
onKet.set_on_rel(G)

#test
s1 = (a+b+c)/sp.sqrt(3)
s2 = (b+c+d)/sp.sqrt(3)

d = sq.Dagger(s1)*s2

#simplify
sq.qapply(d).doit()

In [ ]:
#plot the graph for fun
nx.draw(G, with_labels=True)

In [ ]:
f=sq.Ket("f")
g=sq.Ket("g")
h=sq.TensorProduct(a,b)
l=sq.TensorProduct(b,a)
sq.qapply(sq.Dagger(h)*l).doit()

### Playing with networks

In [ ]:
G = nx.Graph()
G.add_node(1)

In [ ]:
G = nx.Graph()
G.add_nodes_from([1,2,3,4,5])
G.add_edges_from([[1,2],[2,3],[3,4],[4,5],[5,1]])

In [ ]:
%matplotlib inline 
nx.draw(G, with_labels=True)

In [ ]:
3 in G[1]

# Ideal KCBS calculations

## Working Version

In [15]:
class cls_iKCBS:    
    
    def __init__(s,n,verbose=False,thr=10,test=False):
        s.thr=thr
        if verbose==True:
            print("Creating projectors etc")

        cls_iKCBS.n=n
        #j in 0,1,2,3,4
        
        s.ke = [bKet(j) for j in range(3)]    
        s.br = [sq.Dagger(s.ke[j]) for j in range(3)]
        
        s.Πs = []
                
        ### N'd here
        #evaluate the projectors for the ideal case
        for j in range(n):
            s.Πs.append(sp.N(s.eval(j,n)))

        if(test and False):
            print("Testing Πs")
            for i in range(n):
                for k in range(n):
                    print("Inner product of ",i," and ",k," projectors is")
                    temp=0
                    for j in range(3):
                        temp+=sq.qapply(s.br[j]*s.Πs[i]*s.Πs[k]*s.ke[j])
                    print(temp)
                    del temp
                    
   
        #calculate the matrix representation
        s.matΠs = [sq.represent(_) for _ in s.Πs] #I leave this as well to avoid breaking things; TODO: try to remove it
        
        ### N'd here
        #the first two vectors are taken to be k'
        s.kp = [sp.N(s.eval(0,n,True)),sp.N(s.eval(1,n,True))]
        
        #the third is constructed as k0 x k1 (cross product) and then added to the list
        s.kp.append(sp.N(s.cross(s.kp)))
        
        
        #evaluates the conjugate for each element in the list
        s.bp=[sq.Dagger(ikp) for ikp in s.kp]                
        
        if(test):
            #Testing orthogonality
            print("Testing orthogonality of the braP and ketP objects")
            for braP in s.bp:
                for ketP in s.kp:
                    print(sq.qapply(braP*ketP))#sp.N(sq.qapply(braP*ketP)))
        
    
        #the Translation operator
        s.T = s.kp[0]*s.bp[2] + s.kp[1]*s.bp[0] + s.kp[2]*s.bp[1] 
        s.Ti = s.kp[2]*s.bp[0] + s.kp[0]*s.bp[1] + s.kp[1]*s.bp[2] #TODO: Automate this
        
        if(test):
            print("Testing the basic Translation operator")
            for braP in s.bp:
                for ketP in s.kp:
                    #print(braP)
                    print(sq.qapply(braP*s.T*ketP))#sp.N(sq.qapply(braP*ketP)))
                    #print(sp.N(sq.qapply(sp.N(braP)*sp.N(s.T)*sp.N(ketP))))
            display(sq.represent(s.T))

        
        #s.mT = s.mkp[0]*s.mbp[2] + s.mkp[1]*s.mbp[0] + s.mkp[2]*s.mbp[1] 
        #s.mTi = s.mkp[2]*s.mbp[0] + s.mkp[0]*s.mbp[1] + s.mkp[1]*s.mbp[2] #TODO: Automate this
        
        s.matT=sq.represent(s.T)
        
        #Now the abstract ones
        
        #creates n abstract projectors (basically just a bunch of non-commuting symbols)
        s._Πs = sp.symbols('Π_0:%d'%n,commutative=False)
        
        s.daggerDict = {}
        s.sqDict = {}
        for i in range(n):
            s.daggerDict[sq.Dagger(s._Πs[i])]=s._Πs[i]
            s.sqDict[(s._Πs[i])*(s._Πs[i])]=s._Πs[i]
        
        #not very happy because it should come from the graph
        s.gDict = {}
        for i in range(n):
            s.gDict[s._Πs[i]*s._Πs[(i+1)%n]]=sp.Integer(0)
            s.gDict[s._Πs[(i+1)%n]*s._Πs[i]]=sp.Integer(0)
        
        if verbose==True:
            print("Solving a linear system to find the coefficients for expressing the translation operator as a sum of projectors")

        s._T=s.eval_lin_c() #NB: it computes the coefficients into s.c    
        
        ##Debugging
        if(test):
            print("Converting the numerical solution into a usable form")
            numTDict={}
            for i in range(n):
                numTDict[s._Πs[i]]=s.Πs[i]
            #display(s._T)
            numT = s._T.subs(numTDict)
            #display(numT)
            print("Testing the numerically found Translation operator.")
            # for braP in s.bp:
            #     for ketP in s.kp:
            #         print(sq.qapply(braP*numT*ketP)) #sp.N(sq.qapply(braP*ketP)))
            numMatT=sq.represent(numT)
            display(sp.Transpose(numMatT) * numMatT)
            
            display(sq.represent(numT))
            

            
        if verbose==True:
            print("Evaluating the first term of F")

        if(test):
            s.test_swapGate()
            
        if(not test):
            s.Fs = [s.eval_state_F()]        
            # for i in range(n):
            #     print("iteration: ",i)
            #     s.Fs.append(s.Fs[-1]+np.sqrt(5)*s.eval_state_F(s.Πs[i],s._Πs[i]))
            s.F = s.Fs[-1] #last element, the most recent calculation
        
        print("done")
        
    def test_swapGate(s):
        s.TP = sq.TensorProduct 
        dontDie=prune(sp.N(sq.qapply(s._T*s._T)))
        
        _U_=tsimp(s.TP(sp.Integer(1),prune(sp.N(s.kp[0]*s.bp[0]))) + s.TP(s._T,prune(sp.N(s.kp[1]*s.bp[1]))) + s.TP(dontDie,prune(sp.N(s.kp[2]*s.bp[2]))))
        display(_U_)
        _T_=s.TP(sp.Integer(1),prune(sp.N(s.kp[0]*s.bp[0])) + prune(sp.N(s.kp[2]*s.bp[1])) + prune(sp.N(s.kp[1]*s.bp[2])))
        display(_T_)
        _V_=prune(s.TP(s._Πs[0],sp.Integer(1)) + s.TP(s._Πs[1],prune(sp.N(s.Ti))) + s.TP(s._Πs[2],prune(sp.N(s.Ti)*sp.N(s.Ti))))
        display(_V_)
        
    def eval_state_F(s,proj=sp.Integer(1),proj_=sp.Integer(1)):
        s.TP = sq.TensorProduct #just to make naming easy
        dontDie=prune(sp.N(sq.qapply(s._T*s._T)))
                                                                                             
        _U_=tsimp(s.TP(sp.Integer(1),prune(sp.N(s.kp[0]*s.bp[0]))) + s.TP(s._T,prune(sp.N(s.kp[1]*s.bp[1]))) + s.TP(dontDie,prune(sp.N(s.kp[2]*s.bp[2]))))
        _T_=s.TP(sp.Integer(1),prune(sp.N(s.kp[0]*s.bp[0])) + prune(sp.N(s.kp[2]*s.bp[1])) + prune(sp.N(s.kp[1]*s.bp[2])))
        _V_=prune(s.TP(s._Πs[0],sp.Integer(1)) + s.TP(s._Πs[1],prune(sp.N(s.Ti))) + s.TP(s._Πs[2],prune(sp.N(s.Ti)*sp.N(s.Ti))))

        print("Evaluated U,T,V; staying alive")

        _kin_ = s.TP(sp.Integer(1),sp.N(sq.qapply(proj*s.ke[0])))
        _bin_ = s.TP(sp.Integer(1),sp.N(sq.qapply(s.br[0]*proj)))
        
        
        _kout_ = s.TP(sp.Integer(1)*proj_,sp.N(s.kp[0])) #s.TP(sp.Integer(1),sp.N(s.kp[0]))
        _bout_ = s.TP(sp.Integer(1)*proj_,sp.N(s.bp[0])) #s.TP(sp.Integer(1),sp.N(s.bp[0]))
                                                                                             

        print("evaluation stage:")
        F0=tsimp(_bin_ * _T_).subs(s.gDict)
        print("1")
        F1=tsimp(F0 * _U_).subs(s.gDict) #*_V_
        print("2")
        #s.F2=tsimp(F1 * _V_)
        #G00 = tsimp(F1*_V_)
        #print("2.1")
        #display(_U_)
        G00 = tsimp(_U_*_kout_).subs(s.gDict)
        print("2.1")
        G01 = tsimp(_V_*G00).subs(s.gDict)
        print("2.2")
        #G0 = tsimp(_V_*_U_)
        t1=sq.qapply(F1*G01).subs(s.gDict)
        print("2.3")
        
#         G0=tsimp(_V_*_kout_).subs(s.gDict)
#         print("3")
        
#         #display(F1)
#         #display(G0)
#         t1=sq.qapply(F1*G0).subs(s.gDict)
        
        t2=t1.subs(s.daggerDict).subs(s.sqDict) #not sure why the s.daggerDict step is needed;
        
        H0 = tsimp(t2) #tsimp(F1*G0)
        print("4")
        #s.F1 = s._bout_ * _T_ * _U_ * _V_ * s._kout_
        #s.F2 = s.F1 * 
        s.F = tdsimp(H0)
        print("5")
        
        #return tdsimp(sp.expand(s.F * sq.Dagger(s.F)))
        return sp.expand((sq.Dagger(s.F) * s.F).subs(s.daggerDict)).subs(s.sqDict).subs(s.sqDict).subs(s.gDict)
        if verbose==True:
            print("Dinner out is a go.")
        
    def cross(s,vecs):
        ca=[sq.qapply(sq.Dagger(s.ke[i])*vecs[0]).doit() for i in range(3)]
        cb=[sq.qapply(sq.Dagger(s.ke[i])*vecs[1]).doit() for i in range(3)]
        
        res = (ca[2-1]*cb[3-1] - ca[3-1]*cb[2-1])*s.ke[0] + \
             (ca[3-1]*cb[1-1]-ca[1-1]*cb[3-1])*s.ke[1] + \
             (ca[1-1]*cb[2-1] - ca[2-1]*cb[1-1])*s.ke[2]
        
        return res

    #evaluates the matrices and vectors
    def eval(s,j,n,ve=False):
        N=sp.Integer(n)
        J=sp.Integer(j)
        #print(N,j)
        one=sp.Integer(1)
        #print(one)
        α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
        #a1=sp.cos(sp.pi/N)
        #
        #print(α1)
        
        α2=one-α1 #verified
        α3=J * sp.pi * (N-one)/N #verified
        
        vec = sp.sqrt(α1)*s.ke[0] + sp.sqrt(α2)*sp.sin(α3)*s.ke[1] + sp.sqrt(α2)*sp.cos(α3)*s.ke[2]
        
        projector = vec*sq.Dagger(vec)
           
        #matrixprojector = sp.Matrix([0,0])
        #return projector
        
        if ve==False:
            return projector
        else:
            return vec
        
    def eval_lin_c(s,varCount=None):
        cls=s.__class__
        if varCount==None:
            varCount=cls_iKCBS.n*2
        if varCount <= cls.n:
            raise ValueError("number of variables should be at least n")
    
        a=sp.symbols('a0:%d'%varCount)
        #s.b=sp.symbols('b0:%d'%varCount)
        b=[] #family of solutions
        s.c=[] #particular solution with the free variables set to one
        
        #Setting up of constraints
        #If the system is not able to find a solution, consider increasing
        #the number of variables by taking different sets of products or products of three operators
        TfΠ=[a[0]*s.Πs[0]]
        _TfΠ=[a[0]*s._Πs[0]]
        
        for i in range(1,cls.n):
            TfΠ.append(TfΠ[-1] + a[i]*s.Πs[i])
            _TfΠ.append(_TfΠ[-1] + a[i]*s._Πs[i])
        if cls.n != 5:            
            for i in range(cls.n,varCount):
                TfΠ.append(TfΠ[-1] + a[i]*s.Πs[i%cls.n]*s.Πs[(i+2)%cls.n])
                _TfΠ.append(_TfΠ[-1] + a[i]*s._Πs[i%cls.n]*s._Πs[(i+2)%cls.n])
        else:
            TfΠ.append(TfΠ[-1] + a[5]*s.Πs[4]*s.Πs[2] \
                    + a[6]*s.Πs[4]*s.Πs[1] \
                    + a[7]*s.Πs[1]*s.Πs[3] \
                    + a[8]*s.Πs[2]*s.Πs[0])
            _TfΠ.append(_TfΠ[-1] + a[5]*s._Πs[4]*s._Πs[2] \
                    + a[6]*s._Πs[4]*s._Πs[1] \
                    + a[7]*s._Πs[1]*s._Πs[3] \
                    + a[8]*s._Πs[2]*s._Πs[0])
            
            
        LHS=sq.represent(TfΠ[-1])
        #display(_TfΠ)
        # LHS=a[0]*s.matΠs[0]
        # for i in range(1,cls.n):
        #     LHS=LHS+a[i]*s.matΠs[i]
        # for i in range(cls.n,varCount):
        #     LHS=LHS+a[i]*s.matΠs[i%cls.n]*s.matΠs[(i+2)%cls.n]
            
        RHS=s.matT
        if cls.n!=5:
            soln=sp.solve(sp.N(LHS-RHS),a,dict=True)
        else:
            soln=sp.solve(sp.N(LHS-RHS),a[:9],dict=True)
        
        print(soln)
        #print("type:",type(soln[0]))#, " and shape:", soln.shape())
        for key,value in soln[0].items():
            if np.abs(value)<10**(-s.thr):
                soln[0][key]=sp.Integer(0)
        
        #print(soln)
        '''
            #soln=sp.solve([a[0]-1,a[1]-2,a[2]-3,a[3]-4,a[4]-5,a[5]-1,a[6]-1,a[7]-1,a[8]-1,a[9]-1],a,dict=True)

            # sols = solve([t1 + t2 + t3, eq1, eq2], [t1, t2, t3], dict=True)
            # sols[0][t1] # This is t1 in the first solution        
        '''

        #Dictionary to assign one/zero to free variables
        dRem={}
        
        #IMPORTANT: 
        #In sympy, symbols/expressions are unmutable (you can't change them ever);
        #You can save a substituted expression into a new expression (and use a symbol to denote it)
        #To wit: when you substitute, a new expression is produced

        for i in range(varCount):
            #the variables which were evaluated to an expression, assign them to b[i]
            try:
                b.append(soln[0][a[i]])      
            #if the variable was free, assign b[i] to be one
            #and create a dictionary to substitute these free variables
            except:
                b.append(sp.Integer(1))
                dRem[a[i]]=1
        
        #in the solution saved into b[i], substitute for the free variables using the dictionary
        #save the result into the variable c
        s.c = [_.subs(dRem) for _ in b]

        #substitute the solution into the coefficients in Tfπ_p, the operator T as a sum of projectors (and its products)
        dFin={}
        for i in range(varCount):
            dFin[a[i]]=s.c[i]                
        
        return _TfΠ[-1].subs(dFin)
            
    def eval_c(s,varCount=None):
        cls=s.__class__
        if varCount==None:
            varCount=cls_iKCBS.n*2
        if varCount <= cls.n:
            raise ValueError("number of variables should be at least n")
    
        a=sp.symbols('a0:%d'%varCount)
        #s.b=sp.symbols('b0:%d'%varCount)
        b=[] #family of solutions
        s.c=[] #particular solution with the free variables set to one
        
        #Setting up of constraints
        #If the system is not able to find a solution, consider increasing
        #the number of variables by taking different sets of products or products of three operators
        TfΠ=a[0]*s.Πs[0]
        _TfΠ=a[0]*s._Πs[0]
        for i in range(1,cls.n):
            TfΠ=TfΠ + a[i]*s.Πs[i]
            _TfΠ=_TfΠ + a[i]*s._Πs[i]
        for i in range(cls.n,varCount):
            TfΠ=TfΠ + a[i]*s.Πs[i%cls.n]*s.Πs[(i+2)%cls.n]
            _TfΠ=_TfΠ + a[i]*s._Πs[i%cls.n]*s._Πs[(i+2)%cls.n]
            
            
        LHS=sq.represent(TfΠ)
        
        # LHS=a[0]*s.matΠs[0]
        # for i in range(1,cls.n):
        #     LHS=LHS+a[i]*s.matΠs[i]
        # for i in range(cls.n,varCount):
        #     LHS=LHS+a[i]*s.matΠs[i%cls.n]*s.matΠs[(i+2)%cls.n]
            
        RHS=s.matT
        
        soln=sp.solve(sp.N(LHS-RHS),a,dict=True)

        '''
            #soln=sp.solve([a[0]-1,a[1]-2,a[2]-3,a[3]-4,a[4]-5,a[5]-1,a[6]-1,a[7]-1,a[8]-1,a[9]-1],a,dict=True)

            # sols = solve([t1 + t2 + t3, eq1, eq2], [t1, t2, t3], dict=True)
            # sols[0][t1] # This is t1 in the first solution        
        '''

        #Dictionary to assign one/zero to free variables
        dRem={}
        
        #IMPORTANT: 
        #In sympy, symbols/expressions are unmutable (you can't change them ever);
        #You can save a substituted expression into a new expression (and use a symbol to denote it)
        #To wit: when you substitute, a new expression is produced

        for i in range(varCount):
            #the variables which were evaluated to an expression, assign them to b[i]
            try:
                b.append(soln[0][a[i]])      
            #if the variable was free, assign b[i] to be one
            #and create a dictionary to substitute these free variables
            except:
                b.append(sp.Integer(1))
                dRem[a[i]]=1
        
        #in the solution saved into b[i], substitute for the free variables using the dictionary
        #save the result into the variable c
        s.c = [_.subs(dRem) for _ in b]

        #substitute the solution into the coefficients in Tfπ_p, the operator T as a sum of projectors (and its products)
        dFin={}
        for i in range(varCount):
            dFin[a[i]]=s.c[i]                
        
        return _TfΠ.subs(dFin)
        
    #This doesn't work as expected
    def partial_trace(s,M):
        
        s.br_ = [ sq.TensorProduct(sq.IdentityOperator(),_) for _ in s.br]
        s.ke_ = [ sq.TensorProduct(sq.IdentityOperator(),_) for _ in s.ke]
        
        res=0*s.br_[0]*M*s.ke_[0]        #to get the type right!
        for i in range(3):
            for j in range(3):
                res=res+s.br_[i]*M*s.ke_[j]
        #return sq.qapply(res).doit()
        return tsimp(res)
        

In [16]:
iKCBS = cls_iKCBS(5,verbose=True,test=True)

Creating projectors etc
Testing orthogonality of the braP and ketP objects
1.00000000000000
-1.11022302462516e-16
0
-1.11022302462516e-16
1.00000000000000
5.55111512312578e-17
0
5.55111512312578e-17
1.00000000000000
Testing the basic Translation operator
-1.11022302462516e-16
-2.77555756156289e-17
1.00000000000000
1.00000000000000
2.77555756156289e-17
-1.14491749414469e-16
-1.52655665885959e-16
1.00000000000000
-1.17961196366423e-16


⎡0.0126398019966776  0.459505841094722  0.888084803048288 ⎤
⎢                                                         ⎥
⎢0.893751184477034   0.393075688878712  -0.216102575316504⎥
⎢                                                         ⎥
⎣-0.448384941374467  0.796458338403433  -0.405715490875389⎦

Solving a linear system to find the coefficients for expressing the translation operator as a sum of projectors
[{a0: -0.272019649514071, a1: -0.485868271756646, a2: -2.05817102727149, a3: 9.32265207248423e-16, a4: 2.77032771530715, a5: -1.15229372655726, a6: -4.82849874257864, a7: 3.33019067678556, a8: 2.77032771530716}]
Converting the numerical solution into a usable form
Testing the numerically found Translation operator.


⎡  0.999999999999995    -4.82947015711943e-15  -7.7715611723761e-16⎤
⎢                                                                  ⎥
⎢-4.82947015711943e-15           1.0           -7.7715611723761e-16⎥
⎢                                                                  ⎥
⎣-7.7715611723761e-16   -7.7715611723761e-16           1.0         ⎦

⎡0.0126398019966769  0.459505841094722  0.888084803048288 ⎤
⎢                                                         ⎥
⎢0.893751184477031   0.393075688878709  -0.216102575316506⎥
⎢                                                         ⎥
⎣-0.448384941374467  0.796458338403435  -0.40571549087539 ⎦

Evaluating the first term of F


                                                                              
0.447213595499958⋅1⨂ (❘0⟩⋅⟨0❘) + 0.497205787878578⋅1⨂ (❘0⟩⋅⟨2❘) + 0.4972057878

                                                                              
78578⋅1⨂ (❘2⟩⋅⟨0❘) + 0.552786404500042⋅1⨂ (❘2⟩⋅⟨2❘) - 0.121650885505826⋅Π₀⨂ (❘

                                                                              
0⟩⋅⟨0❘) - 0.0794978049906969⋅Π₀⨂ (❘0⟩⋅⟨1❘) + 0.109419341506338⋅Π₀⨂ (❘0⟩⋅⟨2❘) -

                                                                              
 0.0794978049906969⋅Π₀⨂ (❘1⟩⋅⟨0❘) - 0.0519511302532706⋅Π₀⨂ (❘1⟩⋅⟨1❘) + 0.07150

                                                                              
45964286447⋅Π₀⨂ (❘1⟩⋅⟨2❘) + 0.109419341506338⋅Π₀⨂ (❘2⟩⋅⟨0❘) + 0.07150459642864

                                                                            2 
47⋅Π₀⨂ (❘2⟩⋅⟨1❘) - 0.0984176337549742⋅Π₀⨂ (❘2⟩⋅⟨2❘) + 0.00781182724501563⋅Π₀ ⨂

                                  2           

1⨂ (0.0126398019966776⋅❘0⟩⋅⟨0❘ + 0.459505841094722⋅❘0⟩⋅⟨1❘ + 0.888084803048288
⋅❘0⟩⋅⟨2❘ + 0.459505841094722⋅❘1⟩⋅⟨0❘ + 0.786151377757423⋅❘1⟩⋅⟨1❘ - 0.413304238
122399⋅❘1⟩⋅⟨2❘ + 0.888084803048288⋅❘2⟩⋅⟨0❘ - 0.413304238122399⋅❘2⟩⋅⟨1❘ + 0.201
208820245899⋅❘2⟩⋅⟨2❘)

                                                                              
Π₀⨂ 1 + Π₁⨂ (0.0126398019966776⋅❘0⟩⋅⟨0❘ + 0.893751184477034⋅❘0⟩⋅⟨1❘ - 0.448384

                                                                              
941374467⋅❘0⟩⋅⟨2❘ + 0.459505841094722⋅❘1⟩⋅⟨0❘ + 0.393075688878712⋅❘1⟩⋅⟨1❘ + 0.

                                                                              
796458338403433⋅❘1⟩⋅⟨2❘ + 0.888084803048288⋅❘2⟩⋅⟨0❘ - 0.216102575316504⋅❘2⟩⋅⟨1

                                                                              
❘ - 0.405715490875389⋅❘2⟩⋅⟨2❘) + Π₂⨂ (0.0126398019966776⋅❘0⟩⋅⟨0❘ + 0.893751184

                                                                              
477034⋅❘0⟩⋅⟨1❘ - 0.448384941374467⋅❘0⟩⋅⟨2❘ + 0.459505841094722⋅❘1⟩⋅⟨0❘ + 0.393

                                                                              
075688878712⋅❘1⟩⋅⟨1❘ + 0.796458338403433⋅❘1⟩⋅⟨2❘ + 0.888084803048288⋅❘2⟩⋅⟨0❘ -

                                              

done


In [ ]:
#print(iKCBS.c)
#prune(sp.N(sq.qapp\ly(iKCBS.bp[2]*iKCBS.kp[1])))

# display(sq.qapply(iKCBS.F.subs(iKCBS.sqDict)))
# print(iKCBS.sqDict)

In [72]:
# display(sp.simplify(iKCBS.F.subs(iKCBS.sqDict)))
#The orthogonal relations are imposed later using the graph

### Debugging the evaluation of fidelity

Eventually, had to convert all the exact coefficients into numbers; else it was taking too long; it is still rather slow actually.

In [57]:
res=tdsimp(iKCBS.H0)
#display(autoDropDim(sq.qapply(res).doit()))
display(res)

AttributeError: 'cls_iKCBS' object has no attribute 'H0'

In [26]:
res2=iKCBS.F2 #powerDrop(iKCBS.F2)
powow=iKCBS.F2.args[0]
display(powow)
print(isinstance(powow,sp.Pow))
print(powow.exp)
display(sq.qapply(powow.base*powow.base))

display(powerDrop(iKCBS.F2))


#display(res2)

         2
(❘2⟩⋅⟨2❘) 

True
2


❘2⟩⟨2❘

                              ________       _______________________________  
                             ╱ 1   √5       ╱       √5           1            
                            ╱  ─ + ── ⋅    ╱  - ────────── - ────────── + 1 ⋅❘
                          ╲╱   4   4      ╱       ⎛√5   5⎞     ⎛√5   5⎞       
                                         ╱      4⋅⎜── + ─⎟   4⋅⎜── + ─⎟       
  ❘0⟩⟨0❘     √5⋅❘0⟩⟨0❘                 ╲╱         ⎝4    4⎠     ⎝4    4⎠       
────────── + ────────── + ────────────────────────────────────────────────────
  ⎛√5   5⎞     ⎛√5   5⎞                              ________                 
4⋅⎜── + ─⎟   4⋅⎜── + ─⎟                             ╱ √5   5                  
  ⎝4    4⎠     ⎝4    4⎠                            ╱  ── + ─                  
                                                 ╲╱   4    4                  

            ________       _______________________________                    
           ╱ 1   √5       ╱       √5           1   

In [ ]:
res=iKCBS._T_ * iKCBS._U_
display(res)
display(tsimp(res))

In [ ]:
display(iKCBS._U_)

In [ ]:
display(iKCBS._V_)

In [ ]:
display(iKCBS._T_)

In [ ]:
display(iKCBS.F1)

display(tsimp(iKCBS.F1))
#sq.qapply(iKCBS.F1).doit()

In [ ]:

iKCBS=cls_iKCBS(5)

#sp.init_printing()
#sp.pprint(iKCBS.c[0])
for i in range(10):
    display(iKCBS.c[i])
    

display(sp.N(iKCBS.matΠs[1]*iKCBS.matΠs[3]))
display(sp.N(sq.represent(iKCBS.T)))
display(iKCBS._T)
#display(sp.N(iKCBS._T_))
#display(sp.N(iKCBS._U_))
#display(sp.N(iKCBS._V_))
display(sp.N(iKCBS.F))

In [ ]:
#display(iKCBS._U_)

#display(iKCBS.partial_trace(iKCBS._U_))



In [ ]:
g = bKet(0)

Id=sq.IdentityOperator()

display(Id*g)

del Id
del g

In [249]:
g = bKet(0)
h = bKet(1)

TP=sq.TensorProduct
zz=TP(g,g)
zo=TP(g,h)
oo=TP(h,h)
oz=TP(h,g)

display(oz)

ρ = 0.5 * oz*sq.Dagger(oz) + 0.4*oo*sq.Dagger(oo) + 0.1*oz*sq.Dagger(oo)

display(ρ)

display(tsimp(sq.Dagger(oz)*ρ*oz))

display(sq.qapply(sq.Dagger(oz)*ρ*oz))

del TP

❘1⟩⨂ ❘0⟩

0.5⋅❘1⟩⨂ ❘0⟩⋅⟨1❘⨂ ⟨0❘ + 0.1⋅❘1⟩⨂ ❘0⟩⋅⟨1❘⨂ ⟨1❘ + 0.4⋅❘1⟩⨂ ❘1⟩⋅⟨1❘⨂ ⟨1❘

0.500000000000000

         2      2     2                        2     2                        
0.5⋅⟨0❘0⟩ ⋅⟨1❘1⟩ ⋅1⨂ 1  + 0.1⋅⟨0❘0⟩⋅⟨1❘0⟩⋅⟨1❘1⟩ ⋅1⨂ 1  + 0.4⋅⟨0❘1⟩⋅⟨1❘0⟩⋅⟨1❘1⟩

2     2
 ⋅1⨂ 1 

In [263]:
res=0.5 * g + 0.1 * h
display(res)

display(res.args[0])
display(res.args[1])

res.has(sq.Ket)
print(isinstance(res,sp.Add))
print(isinstance(res,sp.Mul))

res2=res.args[0]
print(isinstance(res2,sp.Add))
print(isinstance(res2,sp.Mul))

0.5⋅❘0⟩ + 0.1⋅❘1⟩

0.5⋅❘0⟩

0.1⋅❘1⟩

True
False
False
True


In [ ]:
g = bKet(0)
h = bKet(1)

TP=sq.TensorProduct
zz=TP(g,g)
zo=TP(g,h)
oo=TP(h,h)
oz=TP(h,g)

display(oz)

ρ = 0.5 * oz*sq.Dagger(oz) + 0.4*oo*sq.Dagger(oo) + 0.1*oz*sq.Dagger(oo)

display(ρ)

k=TP(sq.IdentityOperator(),h)


display(tsimp(sq.Dagger(k)*ρ*k))

display(sq.qapply(sq.Dagger(k)*ρ*k))

del TP

In [ ]:
#tsimp=sq.tensorproduct.tensor_product_simp_Mul

#If an inner product evaluates to one, it drops the dimension
#Works only with products and sums for the moment
#No sanity checks were done
#Hopefully, sympy will get updated with this functionality soon anyway



#def(e):
    

Identity=sq.IdentityOperator()
g=bKet(0)
h=bKet(1)

ρ_g=g*sq.Dagger(g)
ρ_h=h*sq.Dagger(h)

# #sq.qapply(Identity*g); seems to work

# #display(sq.TensorProduct(ρ_g,ρ_h))
# print("g")
# g_=sq.TensorProduct(Identity,g*sq.Dagger(g)); display(g_)
# print("h")
# h_=sq.TensorProduct(Identity,h*sq.Dagger(h)); display(h_)

# print("gh")
# display(sq.Dagger(g_)*h_)

f_=sq.TensorProduct(g,g)
f__=sq.TensorProduct(h,g)

#sq.qapply(sq.Dagger(f_)*g_*f_).doit()

print("inner product of ")
display(f_,f__)
#sq.qapply(sq.Dagger(f_)*f_)
#sq.tensorproduct.tensor_product_simp(sq.Dagger(f_)*f__ + sq.Dagger(f__)*f__)
print("yields these two")
display(sq.Dagger(f_)*f__)
display(sq.qapply(sq.Dagger(f_)*f__).doit())

print("inner product of ")
display(f_,f_)
print("yields these two")
res1=sq.Dagger(f_)*f_
display(sq.Dagger(f_)*f_)
display(sq.qapply(sq.Dagger(f_)*f_).doit())


print("Let us try the inner product of these two vectors with the following operator")
op=f_*sq.Dagger(f_)

display(op)
print("which yields these two")
display(sq.qapply(sq.Dagger(f__)*op*f__))
display(sq.qapply(sq.Dagger(f__)*op*f__).doit())
#sq.qapply(sq.Dagger(f_)*h_*sq.Dagger(f__)).doit()


######So far it works well
print("Consider now the tensor product of two operators")
ρ=sq.TensorProduct(ρ_g,ρ_g)
display(ρ)
print("Their inner product with the ket") 
display(f_)
print("yields the following two")
res=sq.qapply(sq.Dagger(f_)*ρ*f_)
display(res)
display(res.doit())

display(tsimp(res))
print("Testing if the constituents are still instances of orthogonal vectors")
print(isinstance(res.args[0].args[0],bBra))
display(res.args[0].args[0],res.args[1])

print("Testing if the constituents are ... same for the original")
print(isinstance(res1.args[0].args[0],bBra))
display(res1.args[0].args[0],res1.args[1])

#sq.qapply(sq.Dagger(g_)*h_).doit() #*ρ_g

#sq.qapply(sq.Dagger(g)*h).doit()
print("Now we try with this ")
ρ2=sq.TensorProduct(ρ_g,ρ_h)
σ=3*ρ
display(σ)
res2=tsimp(sq.qapply(sq.Dagger(f_)*σ*f_))
display(res2)

#res2.has(sq.TensorProduct)
#isinstance(res2,sq.TensorProduct)
#new_args=list(sq.TensorP)
#sq.TensorProduct(*list(res2.args))


# res3=autoDropDim(res2)
# # for _ in res3.args:
# #     display(_)

# display(res3)

In [ ]:


def tensor_product_simp(e, **hints):
    """Try to simplify and combine TensorProducts.

    """
    if isinstance(e, sp.Add):
        return sp.Add(*[tensor_product_simp(arg) for arg in e.args])
    elif isinstance(e, sp.Mul):
        return tensor_product_simp_Mul(e)
    else:
        return e
    
def tensor_product_simp_Mul(e):
    """Simplify a Mul with TensorProducts.

        >>> e
        AxB*CxD
        >>> tensor_product_simp_Mul(e)
        (A*C)x(B*D)

    """
    
    # TODO: This won't work with Muls that have other composites of
    # TensorProducts, like an Add, Commutator, etc.
    # TODO: This only works for the equivalent of single Qbit gates.
    
    if not isinstance(e, sp.Mul):
        return e
    c_part, nc_part = e.args_cnc()
    n_nc = len(nc_part)
    if n_nc == 0:
        return e
#     elif n_nc == 1:
#         if isinstance(nc_part[0], Pow):
#             return  Mul(*c_part) * tensor_product_simp_Pow(nc_part[0])
#         return e
    elif e.has(sq.TensorProduct):
        current = nc_part[0] #I guess this is the 
#         if not isinstance(current, sq.TensorProduct):
#             if isinstance(current, Pow):
#                 if isinstance(current.base, sq.TensorProduct):
#                     current = tensor_product_simp_Pow(current)
#             else:
#                 raise TypeError('TensorProduct expected, got: %r' % current)
        n_terms = len(current.args)
        new_args = list(current.args)
        for next in nc_part[1:]:
            # TODO: check the hilbert spaces of next and current here.
            if isinstance(next, sq.TensorProduct):
                if n_terms != len(next.args):
                    raise QuantumError(
                        'TensorProducts of different lengths: %r and %r' %
                        (current, next)
                    )
                for i in range(len(new_args)):
                    new_args[i] = new_args[i] * next.args[i]
#             else:
#                 if isinstance(next, Pow):
#                     if isinstance(next.base, sq.TensorProduct):
#                         new_tp = tensor_product_simp_Pow(next)
#                         for i in range(len(new_args)):
#                             if( new_args[i] )
#                             new_args[i] = new_args[i] * new_tp.args[i]
#                     else:
#                         raise TypeError('TensorProduct expected, got: %r' % next)
#                 else:
#                     raise TypeError('TensorProduct expected, got: %r' % next)
            current = next
        return Mul(*c_part) * sq.TensorProduct(*new_args)
    elif e.has(Pow):
        new_args = [ tensor_product_simp_Pow(nc) for nc in nc_part ]
        return tensor_product_simp_Mul(Mul(*c_part) * sq.TensorProduct(*new_args))
    else:
        return e

In [ ]:
glaba=sp.Integer(1)
#isinstance(glaba,sp.number)
#type(glaba)
go=bKet(2)


### Debugging the issue with substitutions

In [ ]:

iKCBS.b[1].subs({iKCBS.a[8]:0})

iKCBS.c
#b=sp.symbols('b0:%d'%10)
#b[0].subs(b[0],10)
# gla=sp.Symbol('x'); gla
# gla.subs(gla,10)

#iKCBS.a[8].subs(iKCBS.a[8],0)

In [ ]:
ket1=bKet(0)
ket2=bKet(1)
kets=(ket1,ket2)
bra=[sq.Dagger(iket) for iket in kets]; bra

### Debugging the issue with _eval_args

In [ ]:
class cls_iKCBS:    
    
    def __init__(s,n):
        cls_iKCBS.n=n
        #j in 0,1,2,3,4
        s.ke = [bKet(j) for j in range(3)]    
        s.br = [sq.Dagger(s.ke[j]) for j in range(3)]
        s.πs = []
        s.matπs = [] 
        #poor man's implementation
        for j in range(5):
            s.πs.append(s.proj(j,n))            
            s.matπs.append(s.matRep(s.proj(j,n)))
        s.opT = s.ke[0]*s.br[2] + s.ke[1]*s.br[0] + s.ke[2]*s.br[1]
        s.matT=s.matRep(s.opT)

    #this method is now obsolete    
    def matRep(s,op):
        n=3 #s.__class__.n
        matop = sp.Matrix(np.zeros((n,n)))        
        for i in range(n):
            for j in range(n):
                matop[i,j] = sq.qapply(sq.Dagger(s.ke[i])*op*s.ke[j]).doit() 
        return matop
        
    def proj(s,j,n):
        N=sp.Integer(n)
        J=sp.Integer(j)
        #print(N,j)
        one=sp.Integer(1)
        #print(one)
        α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
        #a1=sp.cos(sp.pi/N)
        #
        #print(α1)
        
        α2=one-α1 #verified
        α3=J * sp.pi * (N-one)/N #verified
        
        vec = sp.sqrt(α1)*s.ke[0] + sp.sqrt(α2)*sp.sin(α3)*s.ke[1] + sp.sqrt(α2)*sp.cos(α3)*s.ke[2]
        
        projector = vec*sq.Dagger(vec)
           
        #matrixprojector = sp.Matrix([0,0])
        return projector
    
    
    def find_coeff_matT(s):
        cls=s.__class__
        a=sp.symbols('a0:%d'%(cls.n*2))
        LHS=a[0]*s.matπs[0]
        for i in range(1,cls.n):
            LHS=LHS+a[i]*s.matπs[i]
        for i in range(cls.n,2*cls.n):
            LHS=LHS+a[i]*s.matπs[i%cls.n]*s.matπs[(i+2)%cls.n]
            
        RHS=s.matT
        return sp.solve(sp.N(LHS-RHS),a)
        #return sp.solve(a[0]-a[2],a)
        
        #print(LHS)
        #for i in range
        

    '''
       def vec(self,j,n):    
           N=sp.Integer(n)
           J=sp.Integer(j)
           #print(N,j)
           one=sp.Integer(1)
           #print(one)
           α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
           #a1=sp.cos(sp.pi/N)
           #
           #print(α1)
        
           α2=one-α1 #verified
           α3=J * sp.pi * (N-one)/N #verified
        
           vec = sp.sqrt(α1)*self.k0 + sp.sqrt(α2)*sp.sin(α3)*self.k1 + sp.sqrt(α2)*sp.cos(α3)*self.k2
        
           #projector = vec*sq.Dagger(vec)
           return vec
    '''

In [ ]:

iKCBS=cls_iKCBS(5)
#to access the vectors 

# #uncomment to test
# sq.qapply(sq.qapply(iKCBS.πs[0]*iKCBS.πs[1]))
# #iKCBS.πs[1]

# sp.N(iKCBS.matπs[2])

# iKCBS.find_coeff_matT()

In [ ]:
iKCBS.find_coeff_opT()

### Debugging

In [ ]:
sq.qapply(sq.Dagger(iKCBS.πs[0])*iKCBS.πs[1]);

#sq.represent(sq.qapply(sq.Dagger(iKCBS.πs[0])*iKCBS.πs[1]),basis=iKCBS.ke)

In [ ]:
#older version without lists
class cls_iKCBS:
    k0=bKet(0)    
    k1=bKet(1)
    k2=bKet(2)
    
    b0=sq.Dagger(k0)
    b1=sq.Dagger(k1)
    b2=sq.Dagger(k2)
    
    πs=[]
    
    def __init__(self,n):
        #j in 0,1,2,3,4
        for j in range(5):
            self.πs.append(self.proj(j,n))
    
    def proj(self,j,n):
        N=sp.Integer(n)
        J=sp.Integer(j)
        #print(N,j)
        one=sp.Integer(1)
        #print(one)
        α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
        #a1=sp.cos(sp.pi/N)
        #
        #print(α1)
        
        α2=one-α1 #verified
        α3=J * sp.pi * (N-one)/N #verified
        
        vec = sp.sqrt(α1)*self.k0 + sp.sqrt(α2)*sp.sin(α3)*self.k1 + sp.sqrt(α2)*sp.cos(α3)*self.k2
        
        projector = vec*sq.Dagger(vec)
        return projector
    '''
       def vec(self,j,n):    
           N=sp.Integer(n)
           J=sp.Integer(j)
           #print(N,j)
           one=sp.Integer(1)
           #print(one)
           α1=sp.cos((sp.pi)/N)/(one+sp.cos(sp.pi/N)) #verified
           #a1=sp.cos(sp.pi/N)
           #
           #print(α1)
        
           α2=one-α1 #verified
           α3=J * sp.pi * (N-one)/N #verified
        
           vec = sp.sqrt(α1)*self.k0 + sp.sqrt(α2)*sp.sin(α3)*self.k1 + sp.sqrt(α2)*sp.cos(α3)*self.k2
        
           #projector = vec*sq.Dagger(vec)
           return vec
    '''

In [ ]:
iKCBS=cls_iKCBS()
sq.qapply(iKCBS.proj(1,5)).doit();

In [ ]:
sp.simplify(sq.qapply(sq.Dagger(iKCBS.k0)*iKCBS.proj(1,5)*iKCBS.proj(2,5)*iKCBS.k0).doit())
#sp.N(sq.qapply(sq.Dagger(iKCBS.k0)*iKCBS.proj(1,5)*iKCBS.proj(2,5)*iKCBS.k0).doit())
#sq.qapply(iKCBS.proj(1,5)*iKCBS.proj(2,5)).doit()

In [ ]:
(sq.Dagger(iKCBS.k0)*iKCBS.k0).doit()

In [ ]:
sp.N(sq.qapply(sq.Dagger(iKCBS.vec(1,5))*iKCBS.vec(1,5)).doit(),5000)

In [ ]:
sp.N(sp.simplify(sq.qapply(sq.Dagger(iKCBS.vec(4,5))*iKCBS.vec(3,5)).doit()))

In [ ]:
sp.N(sq.qapply(sq.Dagger(iKCBS.πs[0])*iKCBS.πs[1]))

## Translation Operator using $\Pi$s

See for the original implementation Self-testing_KCBS_fidelity_bounds

In [18]:

a=sp.symbols('a0:%d'%(cls_iKCBS.n*2))
a[0]
c=0
c=c+a[0]; c


a₀

In [19]:
a=sp.Matrix(np.zeros((2,2)))
a[1,1]=0;a


⎡0.0  0.0⎤
⎢        ⎥
⎣0.0   0 ⎦

In [20]:
a=sqq.IntQubit(2);a

b=sqq.IntQubit(1);
sq.represent(b);
sq.represent(a)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣0⎦

# SDP part (SymPy meets CvxPy)

In [ ]:
N=5

#Ψ=[sp.symbols('Ψ')]
I=[sp.Integer(1)]
A=list(sp.symbols('Π_0:%d'%N,commutative=False))
#A=list(sp.symbols('Π_0:%d'%N,commutative=False))

# display(I)
# display(A)

In [ ]:
objective = iKCBS.F

# objective1=iKCBS.F

# #objective = A[0] + A[1] + A[2] + A[3] + A[4]
# objective2 = A[0]*A[0] + 1e-13*A[1] + 1e-13*A[2] + 1e-13*A[3] + 1e-13*A[4]

# objective=objective
#display(objective);

In [ ]:


L1=A #appends the lists
#print(L1)

L2=[]
for let1 in L1:
    for let2 in L1:
        L2.append(let1*let2)


#These will help simplify A^3 to A when needed
lDepth=2
sqDicts={A[i]**2:A[i] for i in range(len(L1))}


L = I + L1 + L2
L_ = L1 + L2
#print(L)

In [ ]:
#symbol to index dictionary
sTi={}

for i in range(len(L)):
    sTi[L[i]]=i

#print(sTi)

In [ ]:
G = nx.Graph()

#node2 in G[node1] #yes if they are connected by an edge

G = nx.Graph()
G.add_nodes_from(L1)
G.add_edges_from([[L1[i],L1[(i+1)%(N)]] for i in range(N)])

#create a dictionary to simplify expressions later which sets projector products to zero
gDicts={}
for l1 in L1:
    for l2 in L1:
        if l1 in G[l2]:
            gDicts.update({l1*l2:sp.Integer(0)})


%matplotlib inline 
nx.draw(G, with_labels=True)

In [ ]:
#Simplify objective using the orthogonality relations of the graph
display(objective.subs(gDicts))  
objective_ = objective.subs(gDicts)


In [ ]:
#I was trying to directly substitute the matrices corresponding to these using represent but it doesn't seem to work well with non-commutative symbols and matrices

# for i in range(len(A)):
#     dictRep[A[i]]=sp.N(iKCBS.matΠs[i])

#print(dictRep)
test=A[0]*12+A[1]*13
objective_.subs(dictRep)

#X=[] #list(sp.MatrixSymbol('X_0:%d'%N,3,3))
X=[sp.MatrixSymbol('X0',3,3)]
X.append(sp.MatrixSymbol('X1',3,3))
X.append(sp.MatrixSymbol('X2',3,3))
X.append(sp.MatrixSymbol('X3',3,3))
X.append(sp.MatrixSymbol('X4',3,3))

# x1 = MatrixSymbol('x1', 3, 3)
# x2 = MatrixSymbol('x2', 3, 3)
dictMatSymbols={A[i]:X[i] for i in range(5)}

objective_2= objective_.subs(dictMatSymbols)
print(objective_2)

testExp=A[0]*A[1]
display(testExp)
display(testExp.subs(dictMatSymbols))


sp.N(iKCBS.matΠs[0])

dictRep={X[0]:sp.N(iKCBS.matΠs[0])}

objective_3 = objective_2.subs(dictRep)
print(objective_3)

In [ ]:
#print(prune(objective_,10,True))
#temp=prune(sq.qapply(prune(objective_,10,True)))
#print(temp)
#objective.has(sp.Number)

#print(prune(expr=sp.simplify(  )),remNum=True)
#print(prune(expr=objective_,remNum=True))

_=(prune(expr=objective_,remNum=True))
print(prune(expr=_,remNum=True))
del _

## Debugging (checking swap isometry, version 2)

### cEn

In [ ]:
#e.g. if cEn("202") is called, {symName:symVal} is returned where
# symName = A[2]*A[0]*A[2], symVal = the expectation value using iF

def cEn(inVal,sym=A):
    iF=idealFid()
    iVal=[int(inVal[0])] #index of the first
    symName_=[sym[iVal[0]]]  #symbol corresponding to the first
     #blank
    for i_ in inVal[1:]:
        i=int(i_)
        symName_.append(symName_[-1] * sym[i]) #because with symbols, it is a pain, they are immutable so need to use new ones
        iVal.append(i)
    
    rsymName_ = [sym[iVal[-1]]] #symbol corresponding to the last
    riVal = [iVal[-1]]
    
    for i in reversed(iVal[:-1]):
        riVal.append(i)
        rsymName_.append(rsymName_[-1]*sym[i])
    
        
        
    symVal=iF.idealVal(iVal)
    symName = symName_[-1]
    
    rsymVal = iF.idealVal(riVal)
    rsymName = rsymName_[-1]
    
#     print(sym[0])
#     print(symName)
#     print(symVal)
    dictElement = {symName:symVal,rsymName:rsymVal}
#     print(dictElement)
    return dictElement

### State and everything else

In [130]:
oList = ["0", "02", "0202", "0241", "1", "131", "13131", "1313131", "131313131", "1313141", "131314131", "131314141", "1313142", "131314241", "13141", "1314131", "131413131", "1314141", "13142", "1314202", "1314241", "141", "14131", "1413131", "14141", "1414131", "141413131", "1414141", "14142", "1414202", "1414241", "142", "1420", "14202", "14241", "1424131", "142413131", "1424141", "14242", "1424202", "1424241", "2", "20", "202", "2020", "20202", "20241", "2024131", "2024141", "20242", "2024241", "241", "24131", "2413131", "24141", "242", "24202", "24241"]

#["0", "02", "0202", "0241", "1", "131", "13131", "1313131", "131313131", "1313141", "131314131", "131314141", "1313142", "131314241", "13141", "1314131", "131413131", "1314141", "13142", "1314202", "1314241", "141", "14131", "1413131", "14141", "1414131", "141413131", "1414141", "14142", "1414202", "1414241", "142", "1420", "14202", "14241", "1424131", "142413131", "1424141", "14242", "1424202", "1424241", "2", "20", "202", "2020", "20202", "20241", "2024131", "2024141", "20242", "2024241", "241", "24131", "2413131", "24141", "242", "24202", "24241"]

#["0", "02", "0241", "1", "141", "14141", "14142", "142", "1420", "14202", "14241", "14242", "2", "20", "202", "20241", "241", "24141", "242", "24241", "302413", "303", "313", "31413", "3141413", "314203", "3142413", "414", "41414", "4141414", "4141424", "4142024", "41424", "4142414", "4142424", "42024", "4202414", "424", "42414", "4241414", "42424", "4242414"]
# ["0","02","0241","1","141","14141","14142","142","1420","14202","14241","14242","2","20","202","20241","241","24141","242","24241"]
#["0", "02", "0241", "1", "141", "14141", "14142", "142", "1420", "14202", "14241", "14242", "2", "20", "202", "20241", "241", "24141", "242", "24241", "302413", "303", "313", "31413", "3141413", "314203", "3142413", "414", "41414", "4141414", "4141424", "4142024", "41424", "4142414", "4142424", "42024", "4202414", "424", "42414", "4241414", "42424", "4242414"]

#With only state
#= ["0", "02", "0241", "1", "141", "14141", "14142", "142", "1420", "14202", "14241", "14242", "2", "20", "202", "20241", "241", "24141", "242", "24241"]

#State and others
# ["0", "02", "0241", "1", "141", "14141", "14142", "142", "1420", "14202", "14241", "14242", "2", "20", "202", "20241", "241", "24141", "242", "24241", "302413", "303", "313", "31413", "3141413", "314203", "3142413", "414", "41414", "4141414", "4141424", "4142024", "41424", "4142414", "4142424", "42024", "4202414", "424", "42414", "4241414", "42424", "4242414"]
#["0", "02", "0241", "1", "141", "14141", "14142", "142", "1420", "14202", "14241", "14242", "2", "20", "202", "20241", "241", "24141", "242", "24241", "302413", "303", "313", "31413", "3141413", "314203", "3142413", "414", "41414", "4141414", "4141424", "4142024", "41424", "4142414", "4142424", "42024", "4202414", "424", "42414", "4241414", "42424", "4242414"]

oList.sort(key=len)


T=[cEn(oList[0])]             #list of dictionaries; T[0] will have the shortest words, and so on
lastLen=len(oList[0])
i=0
for word in oList[1:]:
    if (lastLen<len(word)):
        lastLen=len(word)
        i+=1
        T.append(cEn(word))
    else:
        T[i].update(cEn(word))
T.reverse()
#print(T)


In [131]:
#print(T)

objectives_=[objective_]

for iT in T:
    #print(iT)
    objectives_.append(objectives_[-1].subs(iT))
    #display(objectives_[-1])

print(objectives_[-1])

1.15278640450004


In [91]:
7.164662/6.0;

In [92]:
1.0002054/1.0;

In [93]:
7.1646296450/(5+np.sqrt(5));

### State and everything else; multi-cursor heavy

In [166]:
#Working on updating this
#done using VS code and multi-cursor
#if it changes, don't do it manually without it!

T1 = {}

T1.update(cEn("0"))
T1.update(cEn("1"))
T1.update(cEn("2"))
T1.update(cEn("2"))

T2= {}


T2.update(cEn("02"))
T2.update(cEn("20"))

T3 = {}

T3.update(cEn("242"))
T3.update(cEn("202"))
T3.update(cEn("242"))
T3.update(cEn("414"))
T3.update(cEn("241"))
T3.update(cEn("424"))
T3.update(cEn("202"))
T3.update(cEn("141"))
T3.update(cEn("142"))


T4 = {}

T4.update(cEn("0241"))
T4.update(cEn("1420"))

T5 = {}


T5.update(cEn("14141"))
T5.update(cEn("14142"))
T5.update(cEn("14202"))
T5.update(cEn("14241"))
T5.update(cEn("14242"))
T5.update(cEn("42024"))
T5.update(cEn("20241"))
T5.update(cEn("42414"))
T5.update(cEn("24141"))
T5.update(cEn("42424"))
T5.update(cEn("24241"))
T5.update(cEn("41414"))
T5.update(cEn("41424"))

T7 = {}

T7.update(cEn("4141414"))
T7.update(cEn("4141424"))
T7.update(cEn("4142024"))
T7.update(cEn("4142414"))
T7.update(cEn("4142424"))
T7.update(cEn("4202414"))
T7.update(cEn("4241414"))
T7.update(cEn("4242414"))

In [168]:
out = objective.subs(gDicts)
out.subs(T7) #.subs(T5).subs(T4).subs(T3).subs(T2).subs(T1)

1.44721359549996⋅Π₀ + 0.50631990099834⋅Π₀⋅Π₂ - 2.54992616372413⋅Π₀⋅Π₂⋅Π₄⋅Π₁ + 
0.0249223594996216⋅Π₁ + 18.9640772918993⋅Π₁⋅Π₄⋅Π₁ - 6.80895705245476⋅Π₁⋅Π₄⋅Π₁⋅
Π₄⋅Π₁ + 0.810241289468414⋅Π₁⋅Π₄⋅Π₁⋅Π₄⋅Π₂ - 3.67134658131212⋅Π₁⋅Π₄⋅Π₂ - 2.54992
616372413⋅Π₁⋅Π₄⋅Π₂⋅Π₀ - 0.337012819708448⋅Π₁⋅Π₄⋅Π₂⋅Π₀⋅Π₂ - 17.7362520995107⋅Π₁
⋅Π₄⋅Π₂⋅Π₄⋅Π₁ + 3.43223718036529⋅Π₁⋅Π₄⋅Π₂⋅Π₄⋅Π₂ + 0.447213595499959⋅Π₂ + 0.5063
1990099834⋅Π₂⋅Π₀ + 0.126024438241778⋅Π₂⋅Π₀⋅Π₂ - 0.337012819708448⋅Π₂⋅Π₀⋅Π₂⋅Π₄⋅
Π₁ - 3.67134658131212⋅Π₂⋅Π₄⋅Π₁ + 0.810241289468414⋅Π₂⋅Π₄⋅Π₁⋅Π₄⋅Π₁ - 0.39367053
5019895⋅Π₂⋅Π₄⋅Π₂ + 3.43223718036529⋅Π₂⋅Π₄⋅Π₂⋅Π₄⋅Π₁ + 2.23606797749979⋅(1.17082
039324994⋅Π₂ + 1.51339645797671⋅Π₂⋅Π₀⋅Π₂ - 1.03064284105144⋅Π₂⋅Π₄⋅Π₂) + 2.2360
6797749979⋅(0.0249223594996216⋅Π₁ + 18.9640772918993⋅Π₁⋅Π₄⋅Π₁ - 6.808957052454
77⋅Π₁⋅Π₄⋅Π₁⋅Π₄⋅Π₁ - 17.7362520995107⋅Π₁⋅Π₄⋅Π₂⋅Π₄⋅Π₁) + 2.23606797749979⋅(-0.97
398512573999⋅Π₃⋅Π₀⋅Π₂⋅Π₄⋅Π₁⋅Π₃ + 0.170820393249937⋅Π₃⋅Π₀⋅Π₃ + 0.00951949424901
162⋅Π₃⋅Π₁⋅Π₃ + 7.24363296022548⋅Π₃⋅Π₁⋅Π₄⋅Π₁⋅Π₃ - 2.6

### Only the state

In [137]:
#With only the state; these were the replacements

T5 = {}

T5.update(cEn("24241"))
T5.update(cEn("24141"))
T5.update(cEn("20241"))
T5.update(cEn("14241"))
T5.update(cEn("14202"))
T5.update(cEn("14142"))
T5.update(cEn("14141"))
T5.update(cEn("14242"))

T4 = {}

T4.update(cEn("0241"))

T3 = {}

T3.update(cEn("141"))
T3.update(cEn("241"))
T3.update(cEn("142"))
T3.update(cEn("242"))
T3.update(cEn("424"))
T3.update(cEn("414"))
T3.update(cEn("202"))

T2 = {}

T2.update(cEn("20"))

T1 = {}

T1.update(cEn("4"))
T1.update(cEn("2"))
T1.update(cEn("1"))
T1.update(cEn("0"))

print(T5,T4,T3,T2,T1)


{Π_2*Π_4*Π_2*Π_4*Π_1: 0.06524758424985277, Π_1*Π_4*Π_2*Π_4*Π_2: 0.06524758424985277, Π_2*Π_4*Π_1*Π_4*Π_1: 0.06524758424985283, Π_1*Π_4*Π_1*Π_4*Π_2: 0.06524758424985282, Π_2*Π_0*Π_2*Π_4*Π_1: 0.06524758424985277, Π_1*Π_4*Π_2*Π_0*Π_2: 0.06524758424985279, Π_1*Π_4*Π_2*Π_4*Π_1: 0.06524758424985279, Π_1*Π_4*Π_1*Π_4*Π_1: 0.06524758424985284} {Π_0*Π_2*Π_4*Π_1: 0.10557280900008412, Π_1*Π_4*Π_2*Π_0: 0.10557280900008413} {Π_1*Π_4*Π_1: 0.17082039324993697, Π_2*Π_4*Π_1: 0.17082039324993692, Π_1*Π_4*Π_2: 0.17082039324993692, Π_2*Π_4*Π_2: 0.17082039324993686, Π_4*Π_2*Π_4: 0.17082039324993686, Π_4*Π_1*Π_4: 0.17082039324993697, Π_2*Π_0*Π_2: 0.17082039324993686} {Π_2*Π_0: 0.276393202250021, Π_0*Π_2: 0.276393202250021} {Π_4: 0.4472135954999579, Π_2: 0.4472135954999579, Π_1: 0.4472135954999579, Π_0: 0.4472135954999579}


In [138]:
#with only the state
out = objective.subs(gDicts)
out.subs(T5).subs(T4).subs(T3).subs(T2).subs(T1)

3.14673624577261

### MetaDebug (Testing list comprehension)

In [87]:
temp1="123"
summe=0
for t in temp1:
    k=int(t)
    summe+=k
print(summe)    

6


In [238]:
temp=["0","1","2","2","02","41","20","142","202","241","242","1420","0241","4241","14141","14142","14202","14242","20241","24141","24241","202","242","414","41414","4141414","4141424","4142024","41424","4142414","4142424","42024","4202414","424","42414","4241414","42424","4242414"]
print(temp)
temp2=temp.sort(key=len)
print(temp,temp2)
print(temp)

['0', '1', '2', '2', '02', '41', '20', '142', '202', '241', '242', '1420', '0241', '4241', '14141', '14142', '14202', '14242', '20241', '24141', '24241', '202', '242', '414', '41414', '4141414', '4141424', '4142024', '41424', '4142414', '4142424', '42024', '4202414', '424', '42414', '4241414', '42424', '4242414']
['0', '1', '2', '2', '02', '41', '20', '142', '202', '241', '242', '202', '242', '414', '424', '1420', '0241', '4241', '14141', '14142', '14202', '14242', '20241', '24141', '24241', '41414', '41424', '42024', '42414', '42424', '4141414', '4141424', '4142024', '4142414', '4142424', '4202414', '4241414', '4242414'] None
['0', '1', '2', '2', '02', '41', '20', '142', '202', '241', '242', '202', '242', '414', '424', '1420', '0241', '4241', '14141', '14142', '14202', '14242', '20241', '24141', '24241', '41414', '41424', '42024', '42414', '42424', '4141414', '4141424', '4142024', '4142414', '4142424', '4202414', '4241414', '4242414']


## Debugging (checking swap isometry)

In [28]:
T_24241_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_24241 = sq.qapply(iKCBS.br[0]*T_24241_*iKCBS.ke[0])
T_24141_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_24141 = sq.qapply(iKCBS.br[0]*T_24141_*iKCBS.ke[0])
T_20241_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[0]*iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_20241 = sq.qapply(iKCBS.br[0]*T_20241_*iKCBS.ke[0])
T_14241_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_14241 = sq.qapply(iKCBS.br[0]*T_14241_*iKCBS.ke[0])
T_14202_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[2]*iKCBS.Πs[0]*iKCBS.Πs[2]))
T_14202 = sq.qapply(iKCBS.br[0]*T_14202_*iKCBS.ke[0])
T_14142_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[2]))
T_14142 = sq.qapply(iKCBS.br[0]*T_14142_*iKCBS.ke[0])
T_14141_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_14141 = sq.qapply(iKCBS.br[0]*T_14141_*iKCBS.ke[0])
T_14242_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[2]))
T_14242 = sq.qapply(iKCBS.br[0]*T_14242_*iKCBS.ke[0])

T_0241_ = sq.qapply(sp.N(iKCBS.Πs[0]*iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_0241 = sq.qapply(iKCBS.br[0]*T_0241_*iKCBS.ke[0])
T_1420_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[2]*iKCBS.Πs[0]))
T_1420 = sq.qapply(iKCBS.br[0]*T_1420_*iKCBS.ke[0])

T_141_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_141 = sq.qapply(iKCBS.br[0]*T_141_*iKCBS.ke[0])
T_241_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[1]))
T_241 = sq.qapply(iKCBS.br[0]*T_241_*iKCBS.ke[0])
T_142_ = sq.qapply(sp.N(iKCBS.Πs[1]*iKCBS.Πs[4]*iKCBS.Πs[2]))
T_142 = sq.qapply(iKCBS.br[0]*T_142_*iKCBS.ke[0])
T_242_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[4]*iKCBS.Πs[2]))
T_242 = sq.qapply(iKCBS.br[0]*T_242_*iKCBS.ke[0])
T_424_ = sq.qapply(sp.N(iKCBS.Πs[4]*iKCBS.Πs[2]*iKCBS.Πs[4]))
T_424 = sq.qapply(iKCBS.br[0]*T_424_*iKCBS.ke[0])
T_414_ = sq.qapply(sp.N(iKCBS.Πs[4]*iKCBS.Πs[1]*iKCBS.Πs[4]))
T_414 = sq.qapply(iKCBS.br[0]*T_414_*iKCBS.ke[0])
T_202_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[0]*iKCBS.Πs[2]))
T_202 = sq.qapply(iKCBS.br[0]*T_202_*iKCBS.ke[0])
T_20_ = sq.qapply(sp.N(iKCBS.Πs[2]*iKCBS.Πs[0]))
T_20 = sq.qapply(iKCBS.br[0]*T_20_*iKCBS.ke[0])
T_02_ = sq.qapply(sp.N(iKCBS.Πs[0]*iKCBS.Πs[2]))
T_02 = sq.qapply(iKCBS.br[0]*T_02_*iKCBS.ke[0])
T_4_ = sq.qapply(sp.N(iKCBS.Πs[4]))
T_4 = sq.qapply(iKCBS.br[0]*T_4_*iKCBS.ke[0])
T_2_ = sq.qapply(sp.N(iKCBS.Πs[2]))
T_2 = sq.qapply(iKCBS.br[0]*T_2_*iKCBS.ke[0])
T_1_ = sq.qapply(sp.N(iKCBS.Πs[1]))
T_1 = sq.qapply(iKCBS.br[0]*T_1_*iKCBS.ke[0])
T_0_ = sq.qapply(sp.N(iKCBS.Πs[0]))
T_0 = sq.qapply(iKCBS.br[0]*T_0_*iKCBS.ke[0])

In [32]:
sanDict1 = {A[0]:T_0, A[1]: T_1, A[2]: T_2, A[4]: T_4}
sanDict2 = {A[0]*A[2]: T_02,A[2]*A[0]: T_20}
sanDict3 = {A[2]*A[0]*A[2]: T_202, A[4]*A[1]*A[4]: T_414, A[4]*A[2]*A[4]: T_424 , A[2]*A[4]*A[2]: T_242 , A[1]*A[4]*A[2]: T_142,A[2]*A[4]*A[1]: T_241 , A[1]*A[4]*A[1]: T_141  }
sanDict4 = {A[0]*A[2]*A[4]*A[1]: T_0241,A[1]*A[4]*A[2]*A[0]: T_1420}
sanDict5 = {A[1]*A[4]*A[2]*A[4]*A[2]: T_14242,A[1]*A[4]*A[1]*A[4]*A[1]: T_14141,A[1]*A[4]*A[1]*A[4]*A[2]: T_14142,A[1]*A[4]*A[2]*A[0]*A[2]: T_14202, A[1]*A[4]*A[2]*A[4]*A[1]: T_14241,A[2]*A[0]*A[2]*A[4]*A[1]: T_20241,A[2]*A[4]*A[1]*A[4]*A[1]: T_24141,A[2]*A[4]*A[2]*A[4]*A[1]: T_24241}

In [33]:
out = objective.subs(gDicts)
out.subs(sanDict5).subs(sanDict4).subs(sanDict3).subs(sanDict2).subs(sanDict1)

1.00020543148321

## Zero list (useless?)

In [52]:
# display(objective.subs(gDicts))
# print(gDicts)
# print(gDicts)
# glabaDict={A[0]*A[1]:sp.Integer(0)}
# print(glabaDict)
# # display(sp.expand(objective).subs(glabaDict))
# # display(sp.expand(objective).args[3].args[1].args[0])
# # print(isinstance(sp.expand(objective).args[3].args[1].args[0],sp.Symbol))
# # print(isinstance(A[0],sp.Symbol))
# obj=sp.expand(objective)
# l=0
# for _ in obj.args:
#     l+=1
#     print("num:",l)
#     display(_.subs(gDicts))
#     for __ in _.args:
#         print("subterm")
#         display(__)
#         if __ == A[0]:
#             print("matched!")
        
#display(objective.args[0])
# expr=12*A[0]*A[1]
# expr2=12*iKCBS._Πs[0]*iKCBS._Πs[1]
# display(expr)
# display(expr.subs(glabaDict))

# display(expr2)
# display(expr2.subs(glabaDict))

# for _ in expr.args:
#     display(_)

# daggerDict={}
# for i in range(5):
#     daggerDict[sq.Dagger(A[i])]=A[i]

In [53]:
zeroDict={}
zeroList=[]
for _ in L1:
    for _2 in L1:
        if _ in G[_2]: #share an edge
            zeroDict[_*_2]=0
            zeroList.append(_*_2)

#print(zeroDict);

## Creating the matrix of symbols (just for show; useless)

In [ ]:
#creating the matrix

matList=[]

for _ in L:
    matList.append([_*_2 for _2 in L])

#print(matList)

sp.Matrix(matList);

## Sympy says hello to Cvxpy

In [59]:
#L is the alphabet, to wit: it is the list of letters
#
X = cp.Variable((len(L),len(L)),symmetric=True)

lI = I[0]

#>> is for matrix inequality
constraints = [X >> 0,X[sTi[lI],sTi[lI]]==1]

#objective
#y = objective.subs({A[0]:X[sTi[A[0]]],A[1]:X[sTi[A[1]]]})
#y = objective.subs({A[0]:sTi[A[0]],A[1]:sTi[A[1]]})


# objective function evaluated (loop over product of all letters)
y = 0

print("Adding constraintst from the objective")
termsUsed = []
for l1 in L:
    print("row of ",l1)
    for l2 in L:
        if l1 != lI or l2 != lI: #this is because if we ask for the coefficient of 1, things get messed up
            term = l1*l2
            
            #the same term can be represented by many elements in the matrix
            #we satisfy ourselves with picking just one of them
            if not(term in termsUsed):
                termsUsed.append(term)                
                eX = X[sTi[l1]][sTi[l2]] #picks the corresponding element from the variable matrix of the SDP library
                #print(eX)
                coeff = sp.N(findCoeff(objective_,l1*l2))
                #display(l1*l2)
                if isinstance(coeff,sp.Number): #evaluates the coff of l1*l2 in the objective
                    #coeff = sp.N(objective.coeff(l1*l2)) #evaluates the coff of l1*l2 in the objective
                    #display(coeff)                    
                    y += coeff*eX
                #else:
                    #print("strange coeff")
                    #display(coeff)
print("done")

print("Adding constraints from the graph")
#Constraints from the graph (these come from single letters)        
for l1 in L1:
    for l2 in L1:
        eX = X[sTi[l1]][sTi[l2]] #picks the corresponding element from the variable matrix of the SDP library
        if l1 in G[l2]:
            constraints += [eX == 0]

print("done")
        #print(coeff,sTi[l1])
        #display(l1)

#constraints from the fact that Π^2=Π
# for l in L1:
#     constraints += [ X[sTi[lI]][sTi[l]] == X[sTi[l]][sTi[l]] ]
#     #constraints to match with experiment

print("Adding constraints from Quantum Mechanics")
dictTerms = {}
#lettersUsed = []
for l1 in L:
    print("Row of ",l1)
    for l2 in L:
        term = l1*l2

        
        tS=[term]
        #display(term)
        for i in range(lDepth+1):
            tS.append(tS[-1].subs(sqDicts))
        simTerm=tS[-1]
        #display(simTerm)

        if not(simTerm in dictTerms):
            dictTerms.update({simTerm:[l1,l2]})            
        else:
            l1_=dictTerms[simTerm][0]
            l2_=dictTerms[simTerm][1]
            #print(l1_,l2_)            
            #print(sTi[l1_],sTi[l2_])
            
            #print(l1,l2)            
            #print(sTi[l1],sTi[l2])
            
            #constraints += [ X[sTi[l1]][sTi[l2]] == X[sTi[l1_]][sTi[l2_]] ] 
            # print(X[sTi[l1]][sTi[l2]])
            # print(X[sTi[l1_]][sTi[l2_]])
            constraints += [ X[sTi[l1]][sTi[l2]] == X[sTi[l1_]][sTi[l2_]] ]

        #Orthogonality from the graph
        if simTerm.subs(gDicts)==sp.Integer(0):
            constraints += [ X[sTi[l1]][sTi[l2]] == 0 ]
        del tS
        
print("done")
# display(y)

# display(objective.args[0])
# display(objective.args[1])


#expr=A[0]*A[0]*A[0]*A[1]*A[0]*A[1]




#         term = l1*l2


#         tS=[term]
#         #display(term)
#         for i in range(lDepth+1):
#             tS.append(tS[-1].subs(sqDicts))
#         simTerm=tS[-1]
#         #display(simTerm)

#         if not(simTerm in dictTerms):
#             dictTerms.update({simTerm:[l1,l2]})            
#         else:
#             l1_=dictTerms[simTerm][0]
#             l2_=dictTerms[simTerm][1]
#             #print(l1_,l2_)            
#             #print(sTi[l1_],sTi[l2_])

#             #print(l1,l2)            
#             #print(sTi[l1],sTi[l2])

#             #constraints += [ X[sTi[l1]][sTi[l2]] == X[sTi[l1_]][sTi[l2_]] ] 
#             # print(X[sTi[l1]][sTi[l2]])
#             # print(X[sTi[l1_]][sTi[l2_]])
#             constraints += [ X[sTi[l1]][sTi[l2]] == X[sTi[l1_]][sTi[l2_]] ]

#         #Orthogonality from the graph
#         if simTerm.subs(gDicts)==sp.Integer(0):
#             constraints += [ X[sTi[l1]][sTi[l2]] == 0 ]
#         del tS



#display(tS[-1])


# powDicts={}
# for i in range(len(L1)):
#     powDicts.update({A[i]**k:A[i] for k in range(2,lDepth+4)[::-1]})


#expr.subs(powDicts)

# display(expr.subs(sqDicts))
# display(expr.subs(powDicts).subs(powDicts))

#expr

#dict={A[i]**k:A[i] for k in range(lDepth)}

#print(powDicts)

#result=expr.subs({A[0]*A[0]:A[1]}); result
#result=sp.refine(expr,)


Adding constraintst from the objective
row of  1
row of  Π_0
row of  Π_1
row of  Π_2
row of  Π_3
row of  Π_4
row of  Π_0**2
row of  Π_0*Π_1
row of  Π_0*Π_2
row of  Π_0*Π_3
row of  Π_0*Π_4
row of  Π_1*Π_0
row of  Π_1**2
row of  Π_1*Π_2
row of  Π_1*Π_3
row of  Π_1*Π_4
row of  Π_2*Π_0
row of  Π_2*Π_1
row of  Π_2**2
row of  Π_2*Π_3
row of  Π_2*Π_4
row of  Π_3*Π_0
row of  Π_3*Π_1
row of  Π_3*Π_2
row of  Π_3**2
row of  Π_3*Π_4
row of  Π_4*Π_0
row of  Π_4*Π_1
row of  Π_4*Π_2
row of  Π_4*Π_3
row of  Π_4**2
done
Adding constraints from the graph
done
Adding constraints from Quantum Mechanics
Row of  1
Row of  Π_0
Row of  Π_1
Row of  Π_2
Row of  Π_3
Row of  Π_4
Row of  Π_0**2
Row of  Π_0*Π_1
Row of  Π_0*Π_2
Row of  Π_0*Π_3
Row of  Π_0*Π_4
Row of  Π_1*Π_0
Row of  Π_1**2
Row of  Π_1*Π_2
Row of  Π_1*Π_3
Row of  Π_1*Π_4
Row of  Π_2*Π_0
Row of  Π_2*Π_1
Row of  Π_2**2
Row of  Π_2*Π_3
Row of  Π_2*Π_4
Row of  Π_3*Π_0
Row of  Π_3*Π_1
Row of  Π_3*Π_2
Row of  Π_3**2
Row of  Π_3*Π_4
Row of  Π_4*Π_0
Row of  

In [60]:
#######################Constraints from the observations

a=1/np.sqrt(5)-0.001
data = [a,a,a,a,a]

j=0
for l1 in L1:    
    constraints += [ X[sTi[l1],sTi[l1]] == data[j] ]
    j+=1
    
del j

In [61]:
# # depth=0
# # depthThresh=20

# def findCoeff(obj,lett):
# #     global depth
    
# #     print("Parent object:")
# #     display(obj)
    
#     if(obj==None):
# #         print("coefficent is zero")
#         return 0
#     elif not (isinstance(obj,sp.Mul) or isinstance(obj,sp.Add)):        
#         #the coefficient may be one but we couldn't see it earlier
#         return obj.coeff(lett)
#     else:
#         #try to find the coefficient directly        
#         result=obj.coeff(lett)
#         #Did not work? 
#         if(result==0):
#             #try for each segment
#             for _ in obj.args:
# #                 print("child:")
# #                 display(_)
#                 result=_.coeff(lett)
# #                 print("coefficient of child:",result)
                
#                 #still did not work?
#                 if(result==0):
#                     #try recursing
#                     result = findCoeff(_,lett)    
                
#                 #found? Stop searching
#                 if(result!=0):
#                     break
#                     #return result

#         #return whatever was found | worked
#         return result


# # def findCoeff2(obj,lett):   
# #     global depth
# # #     result=obj.coeff(lett)
# # #     if result!=0:
# # #         print("nonzero")
# # #         print(result)
# # #         return result
# # #     else:
# #     print(obj)
# #     for _ in obj.args:
# #         display(_)
# #         if depth>depthThresh:
# #             print("depth exceeded")
# #             return 0
# #         elif _==None:
# #             print("coeffecient is zero")
# #             return 0
# #         else:
# #             depth+=1
# #             result=obj.coeff(lett)
# #             #print(depth)
# #             if result!=0:
# #                 print("result was:",result)
# #             else:
# #                 print("recursing")
# #                 print("_ is",_)
# #                 #if isinstance(_,sp.Mul):
# #                 result=findCoeff(_,lett)
            
# #             return result
# # #     if 
# # #                 else:
# # #                     return 0
# # #         else:
# # #             print("depth exceeded")
# # #             return 0
            


In [62]:
probMin = cp.Problem(-3*cp.Maximize(-y),constraints)
probMax = cp.Problem(-3*cp.Maximize(y),constraints)
#prob.solve()
#prob.solve(solver=cp.CVXOPT,verbose=True,**{'':ldl,'refinement':20,'show_progress':True,'feastol':10e-10,'reltol':10e-10,'abstol':10e-10})
try: 
    print("max:",probMax.solve(verbose=True))
except:
    print("max:gla")
try:
    print("min:",-probMin.solve(verbose=True))
except:
    print("min:ba")

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 2237
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 496, constraints m = 1727
Cones:	primal zero / dual free vars: 1231
	sd vars: 496, sd blks: 1
Setup time: 5.48e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 7.05e+19  2.84e+20  1.00e+00 -3.00e+21  2.70e+22  9.49e+21  4.62e-03 
    60| 1.21e+14  3.10e+14  9.06e-03 -2.76e+21 -2.71e+21  4.95e+19  4.62e-02 
----------------------------------------------------------------------------
Sta

In [164]:
import cvxopt as co



## Inputs